In [ ]:
import numpy as np
import pydicom as pydicom
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tempfile
from pydicom.dataset import Dataset, FileDataset
from pydicom.datadict import DicomDictionary, keyword_dict
from pydicom.uid import generate_uid

import SimpleITK as sitk
import datetime, time
import os
import sys
import shutil
import csv

In [ ]:

source_dir = r'F:\ChestData'
target_dir = r'D:\Pathologies'

info_pd = pd.read_csv('CASES_ALL.csv', delimiter=',')
print('Total Cases: ', len(info_pd))
count =0 
for idx, rows in  info_pd.iterrows():
    print(str (info_pd['ImageID'][idx] ), str(info_pd['Unnamed: 0'][idx]), str(info_pd['ImageDir'][idx]) )
    FileName = str(info_pd['ImageID'][idx])
    parentFolder = str(info_pd['ImageDir'][idx])
    
    #Read the Image file
    ImageSourcePath =  os.path.join(source_dir,parentFolder,FileName)
    im = sitk.ReadImage(ImageSourcePath)
    NameParts = FileName.split('.')
    NamePartWithoutExt = NameParts[0]
    
    print(im.GetNumberOfComponentsPerPixel())
    if im.GetNumberOfComponentsPerPixel()>1:
        im = sitk.VectorIndexSelectionCast(im,0,sitk.sitkUInt16)
    else:
        im = sitk.Cast(im, sitk.sitkUInt16)
    print(im.GetPixelIDTypeAsString())
    
    ### NOW CONVERT TO DICOM FORMAT ###
    fn = NamePartWithoutExt+'.dcm'
    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.1'
    file_meta.MediaStorageSOPInstanceUID = "1.2.3"
    file_meta.ImplementationClassUID = "1.2.3.4"
    dcm = FileDataset(fn, {},file_meta=file_meta, preamble=b"\0" * 128)

  
    
    dcm.ImageType=['ORIGINAL', 'PRIMARY']
    dcm.SOPClassUID=  '1.2.840.10008.5.1.4.1.1.1.1' #  ref.SOPClassUID
    dcm.SOPInstanceUID=generate_uid()
    dcm.file_meta.MediaStorageSOPClassUID = dcm.SOPClassUID
    dcm.file_meta.MediaStorageSOPInstanceUID = generate_uid()
    dcm.Modality          = str (info_pd['Modality_DICOM'][idx])
    dcm.Manufacturer      = 'Unknown'
    dcm.PatientName       = 'Unknown'
    dcm.PatientID         = str (info_pd['PatientID'][idx])
    dcm.ViewPosition      = str (info_pd['ViewPosition_DICOM'][idx] )



    ###  Calculte Age: StudyDate - BirthYear #### StudyDate_DICOM (YYMMDD)                             
    dcm.PatientBirthDate = str (info_pd['PatientBirth'][idx] )
    BirthDate  =  int (info_pd['PatientBirth'][idx] )
    StudyDate  =  int (info_pd['StudyDate_DICOM'][idx] )
    PatientAge = str(StudyDate//10000 - BirthDate )
    #####################################################
    dcm.PatientAge        = PatientAge
    dcm.PatientSex       = str (info_pd['PatientSex_DICOM'][idx])
    import random 
    dcm.AccessionNumber=str(random.randint(1e9,1e10)*100000+66666)
    dcm.ReferringPhysicianName = 'Unknown'
    dcm.StudyID = str (info_pd['StudyID'][idx])
    dcm.BodyPartExamined='CHEST'
    #dcm.ImagerPixelSpacing=[str(sizeInfo[2]),str(sizeInfo[3]) ]

    #cm.PixelSpacing = [str(sizeInfo[0] * sizeInfo[2] / im.GetSize()[0]),
    #                   str(sizeInfo[1] * sizeInfo[3] / im.GetSize()[1])]
    #cm.StudyDescription='Reconstructed Chest XRay'
    #cm.SeriesDescription='Reconstructed Chest XRay'

    dcm.StudyInstanceUID=generate_uid()
    dcm.SeriesInstanceUID=generate_uid()
    dcm.SeriesNumber=  str(info_pd['ImageDir'][idx]) #'1' #Folder ID
    dcm.InstanceNumber= str(info_pd['Unnamed: 0'][idx]) #'1' # Column 0
    dcm.PatientOrientation=['L', 'F']
    dcm.SamplesperPixel=1
    dcm.PhotometricInterpretation='MONOCHROME2'
    dcm.Rows=im.GetSize()[1]
    dcm.Columns=im.GetSize()[0]
    #dcm.PixelSpacing=[str(sizeInfo[0]*sizeInfo[2]/im.GetSize()[0]), str(sizeInfo[1]*sizeInfo[3]/im.GetSize()[1])]
    dcm.BitsAllocated=16
    dcm.BitsStored= info_pd['BitsStored_DICOM'][idx] 
    dcm.HighBit= int(info_pd['BitsStored_DICOM'][idx])-1 # BitsStored -1
    dcm.PixelRepresentation=info_pd['PixelRepresentation_DICOM'][idx]
    dcm.SamplesPerPixel=1
    dcm.StudyDate= str (info_pd['StudyDate_DICOM'][idx])
    dcm.StudyTime= 'Unknown'
     
    
    dcm.PixelData=sitk.GetArrayFromImage(im).tostring()


    # Set the transfer syntax
    dcm.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    dcm.is_little_endian = True
    dcm.is_implicit_VR = False
    
    #Save To Our Destination
    name = idx
    savingName = os.path.join(target_dir,name+'.dcm')
    dcm.save_as(savingName)
    count+=1



In [ ]:
print('Conversion of .png to .dcm is done successfully. :) ')